In [54]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text \
    import text_to_word_sequence
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

from unidecode import unidecode

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

#MAX_WORDS = 1000
#pad=30
#EMBEDDING_WIDTH = 512

EPOCHS = 32
BATCH_SIZE = 256
#INPUT_FILE_NAME = 'F:/DEEP_LEARNING/FINAL/ESTANCIA/Prueba_2.csv'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 1


In [2]:
file = pd.read_csv("C:/Users/IN335255/Documents/OSCAR/PRUEBA/EST/ABRIL_2022/PRUEBA_MAYO_2022.csv", encoding='latin-1', dtype="string")
file = file.dropna()

In [3]:
a = np.zeros((10000,1))
a[0] = 1
b = np.ones((10000,1))/10000
c = sum(a*np.log(b))
c

array([-9.21034037])

Revisando vacíos

In [4]:
# Checking for NaN values.
print("Reviso vacíos\n", file.isna().sum())

Reviso vacíos
 TEXTO    0
dtype: int64


In [5]:
file

,TEXTO
0,# por ciento de nomina .isr retenido por sueld...
1,#% de nomina
2,#% de nomina
3,%3 sobre nomina
4,& sobre nomina
...,...
64777,ZONAS TURISTICAS CERRADAS
64778,ZOOM
64779,ZOOM
64780,ZOOM


In [6]:
file = file.loc[file["TEXTO"] != "0"]
file = file.loc[file["TEXTO"] != "98"]
file = file.loc[file["TEXTO"] != "99"]
file = file.loc[file["TEXTO"] != "NC"]
file = file.loc[file["TEXTO"] != "NS"]
file = file.loc[file["TEXTO"] != "NC NS"]
file = file.loc[file["TEXTO"] != "nc"]
file = file.loc[file["TEXTO"] != "ns"]
file = file.loc[file["TEXTO"] != "nc ns"]

In [7]:
file

,TEXTO
0,# por ciento de nomina .isr retenido por sueld...
1,#% de nomina
2,#% de nomina
3,%3 sobre nomina
4,& sobre nomina
...,...
64777,ZONAS TURISTICAS CERRADAS
64778,ZOOM
64779,ZOOM
64780,ZOOM


In [8]:
#file_train, file_test = train_test_split(file, test_size=0.1)

In [9]:
file

,TEXTO
0,# por ciento de nomina .isr retenido por sueld...
1,#% de nomina
2,#% de nomina
3,%3 sobre nomina
4,& sobre nomina
...,...
64777,ZONAS TURISTICAS CERRADAS
64778,ZOOM
64779,ZOOM
64780,ZOOM


In [10]:
import re
def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = re.sub(r"\+", " plus ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)
   
    text = text.split()

    return text

In [11]:
#file_train.TEXTO = file_train.TEXTO.apply(lambda x: text_to_word_list(x, unidecode))
#file_test.TEXTO = file_test.TEXTO.apply(lambda x: text_to_word_list(x, unidecode))

file.TEXTO = file.TEXTO.apply(lambda x: text_to_word_list(x, unidecode))

In [12]:
file

,TEXTO
0,"[por, ciento, de, nomina, isr, retenido, por, ..."
1,"[de, nomina]"
2,"[de, nomina]"
3,"[3, sobre, nomina]"
4,"[sobre, nomina]"
...,...
64777,"[zonas, turisticas, cerradas]"
64778,[zoom]
64779,[zoom]
64780,[zoom]


In [13]:
file = file.reset_index(drop=True)
file

,TEXTO
0,"[por, ciento, de, nomina, isr, retenido, por, ..."
1,"[de, nomina]"
2,"[de, nomina]"
3,"[3, sobre, nomina]"
4,"[sobre, nomina]"
...,...
61511,"[zonas, turisticas, cerradas]"
61512,[zoom]
61513,[zoom]
61514,[zoom]


In [14]:
# Create training examples.
fragments = []
targets = []

#for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
#    fragments.append(text[i: i + WINDOW_LENGTH])
#    targets.append(text[i + WINDOW_LENGTH])

for i in range(0, len(file)):
    fragments.append(file.TEXTO[i][0: len(file.TEXTO[i])-1])
    
    
for i in range(0, len(file)):    
    targets.append(file.TEXTO[i][len(file.TEXTO[i])-1])

In [15]:
MAX_WORDS = 150
pad=30
EMBEDDING_WIDTH = 128

In [16]:
file_2=[]

for i in range(0, len(file)):
    file_2.append(file.TEXTO[i])

In [17]:
# Convert to indices.
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='UNK')
# Aqui asigna indices a todo el corpus
#tokenizer.fit_on_texts(text)
tokenizer.fit_on_texts(file_2)

# Aqui aqui asigna indices a las palabras de cada cadena 
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)

#data_tensor = tf.ragged.constant(fragments_indexed)

# Convert to appropriate input and output formats.
#X = np.array(fragments_indexed, dtype=np.int)

X = np.array(fragments_indexed, dtype=object)
y = np.zeros((len(targets_indexed), MAX_WORDS))
for i, target_index in enumerate(targets_indexed):
    y[i, target_index] = 1
    
    
print("Number of words in the dictionary", len(tokenizer.word_index))
print(list(tokenizer.word_index.items())[:10])

Number of words in the dictionary 14886
[('UNK', 1), ('de', 2), ('y', 3), ('la', 4), ('los', 5), ('en', 6), ('no', 7), ('falta', 8), ('a', 9), ('mas', 10)]


In [23]:
# REVISA distribución: calcula 
y_hist, x_hist = np.histogram(sorted(tokenizer.word_counts.values()), 100)
print(x_hist)
print(y_hist)

[1.000000e+00 2.515800e+02 5.021600e+02 7.527400e+02 1.003320e+03
 1.253900e+03 1.504480e+03 1.755060e+03 2.005640e+03 2.256220e+03
 2.506800e+03 2.757380e+03 3.007960e+03 3.258540e+03 3.509120e+03
 3.759700e+03 4.010280e+03 4.260860e+03 4.511440e+03 4.762020e+03
 5.012600e+03 5.263180e+03 5.513760e+03 5.764340e+03 6.014920e+03
 6.265500e+03 6.516080e+03 6.766660e+03 7.017240e+03 7.267820e+03
 7.518400e+03 7.768980e+03 8.019560e+03 8.270140e+03 8.520720e+03
 8.771300e+03 9.021880e+03 9.272460e+03 9.523040e+03 9.773620e+03
 1.002420e+04 1.027478e+04 1.052536e+04 1.077594e+04 1.102652e+04
 1.127710e+04 1.152768e+04 1.177826e+04 1.202884e+04 1.227942e+04
 1.253000e+04 1.278058e+04 1.303116e+04 1.328174e+04 1.353232e+04
 1.378290e+04 1.403348e+04 1.428406e+04 1.453464e+04 1.478522e+04
 1.503580e+04 1.528638e+04 1.553696e+04 1.578754e+04 1.603812e+04
 1.628870e+04 1.653928e+04 1.678986e+04 1.704044e+04 1.729102e+04
 1.754160e+04 1.779218e+04 1.804276e+04 1.829334e+04 1.854392e+04
 1.879450e

In [24]:
X_final = tf.keras.preprocessing.sequence.pad_sequences(fragments_indexed,
                                                    #value=word_index["<PAD>"],
                                                    padding='post',
                                                    maxlen=pad)

In [25]:
X_final

array([[16,  1,  2, ...,  0,  0,  0],
       [ 2,  0,  0, ...,  0,  0,  0],
       [ 2,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  0]])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y, test_size=0.1)

In [27]:
X_train.shape

(55364, 30)

In [28]:
X_test.shape

(6152, 30)

In [29]:
y_train.shape

(55364, 150)

In [30]:
y_test.shape

(6152, 150)

In [31]:
training_model = Sequential()
training_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, input_length=None))
training_model.add(LSTM(128, return_sequences=True,
                        dropout=0.2, recurrent_dropout=0.2))
training_model.add(LSTM(128, dropout=0.2,
                        recurrent_dropout=0.2))
training_model.add(Dense(128, activation='relu'))
training_model.add(Dense(MAX_WORDS, activation='softmax'))
training_model.compile(loss='categorical_crossentropy',
                       optimizer='adam')
training_model.summary()
history = training_model.fit(X_train, y_train, validation_split=0.05,
                             batch_size=BATCH_SIZE,
                             epochs=EPOCHS, verbose=2,
                             shuffle=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         19200     
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         131584    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 150)               19350     
Total params: 318,230
Trainable params: 318,230
Non-trainable params: 0
_________________________________________________________________
Epoch 1/32
206/206 - 82s - loss: 3.2078 - val_loss: 2.7183
Epoch 2/32
206/206 - 71s - loss: 2.4087 - val_loss: 2.209

In [32]:
training_model.evaluate(X_train, y_train)

1731/1731 [==============================] - 20s 11ms/step - loss: 1.5715


1.5715274810791016

In [33]:
training_model.evaluate(X_test, y_test)

193/193 [==============================] - 2s 11ms/step - loss: 1.7619


1.7618924379348755

In [34]:
inference_model = Sequential()
inference_model.add(Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS,
    mask_zero=True, batch_input_shape=(1, 1)))

#metemos stateful true y eso hace que metamos batch input shape
inference_model.add(LSTM(128, return_sequences=True,
                         dropout=0.2, recurrent_dropout=0.2,
                         stateful=True))
inference_model.add(LSTM(128, dropout=0.2,
                         recurrent_dropout=0.2, stateful=True))
inference_model.add(Dense(128, activation='relu'))
inference_model.add(Dense(MAX_WORDS, activation='softmax'))
weights = training_model.get_weights()
inference_model.set_weights(weights)

In [48]:
# Provide beginning of sentence and
# predict next words in a greedy manner
first_words = ['covid','pandemia']
first_words_indexed = tokenizer.texts_to_sequences(
    first_words)
inference_model.reset_states()
predicted_string = ''
# Feed initial words to the model.
for i, word_index in enumerate(first_words_indexed):
    x = np.zeros((1, 1), dtype=np.int)
    x[0][0] = word_index[0]
    predicted_string += first_words[i]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
# Predict PREDICT_LENGTH words.
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts(
        [[new_word_index]])
    x[0][0] = new_word_index
    predicted_string += word[0]
    predicted_string += ' '
    y_predict = inference_model.predict(x, verbose=0)[0]
print(predicted_string)

covid pandemia UNK 


In [36]:
# Obtenemos los embeddings
embeddings = training_model.layers[0].get_weights()[0]
# buscamos similitudes

lookup_words = ['pago', 'economia', 'covid', 'venta', 'compra', 'credito', 'nomina', 'trabajo', 'falta']

#lookup_words = ['covid']

for lookup_word in lookup_words:
    lookup_word_indexed = tokenizer.texts_to_sequences([lookup_word])
    print('words close to:', lookup_word)
    lookup_embedding = embeddings[lookup_word_indexed[0]]
    word_indices = {}
    
# Calculamos distancias
    for i, embedding in enumerate(embeddings):
        distance = np.linalg.norm(embedding - lookup_embedding)
        word_indices[distance] = i

    #Imprimimos
    for distance in sorted(word_indices.keys())[:5]:
        word_index = word_indices[distance]
        word = tokenizer.sequences_to_texts([[word_index]])[0]
        print(word + ': ', distance)
    print('')


words close to: pago
pago:  0.0
pagos:  1.4959466
pagar:  1.7248011
con:  1.815253
economico:  1.8581797

words close to: economia
economia:  0.0
economica:  1.001647
economico:  1.1113858
insumos:  1.1215669
personal:  1.1222191

words close to: covid
covid:  0.0
mantenimiento:  1.0603114
obras:  1.107882
limpieza:  1.408184
produccion:  1.4324253

words close to: venta
venta:  0.0
elaboracion:  1.3964925
covid:  1.4618841
mantenimiento:  1.5812467
gobierno:  1.6071922

words close to: compra
compra:  0.0
general:  1.7225503
nominas:  1.8751483
otro:  1.8784628
hay:  1.9193175

words close to: credito
credito:  0.0
nomina:  1.0340422
proyectos:  1.169216
otro:  1.186212
proveedores:  1.1869321

words close to: nomina
nomina:  0.0
credito:  1.0340422
e:  1.0646582
otro:  1.0787495
gobierno:  1.1082321

words close to: trabajo
trabajo:  0.0
trabajar:  1.3293036
obra:  1.5409615
limpieza:  1.5856767
gente:  1.5922476

words close to: falta
falta:  0.0
nm:  1.0614479
UNK:  1.0893238
tener

In [55]:
modelo_embeddings = Model(inputs=training_model.input, outputs=training_model.layers[0].output)



In [57]:
modelo_embeddings.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_input (InputLayer) [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         19200     
Total params: 19,200
Trainable params: 19,200
Non-trainable params: 0
_________________________________________________________________


In [ ]:
z_text2 = sub_model.predict(sentences2)
df_z_text2 = pd.concat([dataframe['nameID'], pd.DataFrame(z_text2)], axis=1)
df_z_text2.to_pickle("z_textVinVL_64.pkl")

df_z_descriptions = pd.read_pickle('z_textVinVL_64.pkl')

